In [19]:
import pandas as pd
MP_20_train = pd.read_csv("cdvae/data/mp_20/train.csv", index_col=0)

In [20]:
from pymatgen.io.cif import CifParser
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

In [21]:
structure = CifParser.from_str(MP_20_train.iloc[1].cif).get_structures()[0]

In [22]:
MP_20_train.iloc[1]

material_id                                                          mp-974729
formation_energy_per_atom                                            -0.314759
band_gap                                                                   0.0
pretty_formula                                                      Nd(Al2Cu)4
e_above_hull                                                               0.0
elements                                                    ['Al', 'Cu', 'Nd']
cif                          # generated using pymatgen\ndata_Nd(Al2Cu)4\n_...
spacegroup.number                                                          139
Name: 19480, dtype: object

In [23]:
len(structure)

13

In [24]:
analyzer = SpacegroupAnalyzer(structure)

In [35]:
sym_structure = analyzer.get_symmetrized_structure()

In [36]:
sym_structure

SymmetrizedStructure
Full Formula (Nd1 Al8 Cu4)
Reduced Formula: Nd(Al2Cu)4
Spacegroup: I4/mmm (139)
abc   :   5.199118   6.809864   6.809864
angles:  81.620960  67.558845  67.558845
Sites (13)
  #  SP           a         b         c  Wyckoff
---  ----  --------  --------  --------  ---------
  0  Nd    0         0         0         1a
  1  Al    0.721222  0.778778  0.778778  4j
  2  Al    0.34894   0.65106   0.65106   4i
  3  Cu    0.5       0.5       0         4f

In [26]:
len(sym_structure.equivalent_sites)

4

In [27]:
analyzer.get_symmetry_dataset().keys()

dict_keys(['number', 'hall_number', 'international', 'hall', 'choice', 'transformation_matrix', 'origin_shift', 'rotations', 'translations', 'wyckoffs', 'site_symmetry_symbols', 'crystallographic_orbits', 'equivalent_atoms', 'primitive_lattice', 'mapping_to_primitive', 'std_lattice', 'std_types', 'std_positions', 'std_rotation_matrix', 'std_mapping_to_primitive', 'pointgroup'])

In [28]:
analyzer.get_symmetry_dataset()['crystallographic_orbits']

array([0, 1, 1, 3, 3, 3, 3, 1, 1, 9, 9, 9, 9], dtype=int32)

In [29]:
analyzer.get_symmetry_dataset()['site_symmetry_symbols']

['4/mmm',
 'm2m.',
 'm2m.',
 'm2m.',
 'm2m.',
 'm2m.',
 'm2m.',
 'm2m.',
 'm2m.',
 '..2/m',
 '..2/m',
 '..2/m',
 '..2/m']

In [30]:
analyzer.get_symmetry_dataset()['wyckoffs']

['a', 'j', 'j', 'i', 'i', 'i', 'i', 'j', 'j', 'f', 'f', 'f', 'f']

In [31]:
sites = dict(((sym_structure.wyckoff_symbols[i], sym_structure.equivalent_sites[i][0].frac_coords) for i in range(len(sym_structure.equivalent_sites))))

In [32]:
sites

{'1a': array([0., 0., 0.]),
 '4j': array([0.721222, 0.778778, 0.778778]),
 '4i': array([0.34894, 0.65106, 0.65106]),
 '4f': array([0.5, 0.5, 0. ])}

In [33]:
len(sym_structure)

13

In [34]:
sym_structure.equivalent_sites

[[PeriodicSite: Nd (0.0, 0.0, 0.0) [0.0, 0.0, 0.0]],
 [PeriodicSite: Al (7.817, 6.734, 1.554) [0.7212, 0.7788, 0.7788],
  PeriodicSite: Al (6.483, 3.238, -0.6977) [0.5, 0.2212, 0.7788],
  PeriodicSite: Al (2.347, 2.245, 0.2946) [0.2788, 0.2212, 0.2212],
  PeriodicSite: Al (3.682, 5.741, 2.546) [0.5, 0.7788, 0.2212]],
 [PeriodicSite: Al (6.1, 4.489, 1.803) [0.3489, 0.6511, 0.6511],
  PeriodicSite: Al (4.984, 1.567, 1.038) [5.794e-17, 0.3489, 0.6511],
  PeriodicSite: Al (3.467, 2.923, 2.795) [4.52e-17, 0.6511, 0.3489],
  PeriodicSite: Al (4.065, 4.489, 0.04528) [0.6511, 0.3489, 0.3489]],
 [PeriodicSite: Cu (1.714, 4.489, 1.42) [0.5, 0.5, 0.0],
  PeriodicSite: Cu (7.594, 2.245, 1.42) [5.551e-17, 0.5, 1.0],
  PeriodicSite: Cu (5.082, 4.489, 4.329) [1.452e-17, 1.0, 0.5],
  PeriodicSite: Cu (5.939, 6.734, 3.337) [0.5, 1.0, 0.5]]]

In [37]:
from pyxtal import pyxtal

In [39]:
my_crystal = pyxtal()
#my_crystal.from_random(dim=3, group="I4/mmm", species=['Nd', 'Al', 'Cu'], numIons=[2, 16, 8])
my_crystal.from_random(dim=3, group="I4/mmm", species=['Nd', 'Al', 'Cu'], numIons=[1, 8, 4])

Comp_CompatibilityError: Compoisition [1 8 4] not compatible with symmetry 139

In [100]:
my_crystal_pymatgen = my_crystal.to_pymatgen()

In [102]:
analyzer = SpacegroupAnalyzer(my_crystal_pymatgen)

In [104]:
analyzer.get_symmetrized_structure()

SymmetrizedStructure
Full Formula (Nd2 Al16 Cu8)
Reduced Formula: Nd(Al2Cu)4
Spacegroup: I4/mmm (139)
abc   :   6.471440   6.471440  11.870218
angles:  90.000000  90.000000  90.000000
Sites (26)
  #  SP           a         b          c  Wyckoff
---  ----  --------  --------  ---------  ---------
  0  Al    0         0.5       -0         4c
  1  Al    0         0          0.149988  4e
  2  Al    0.25      0.25       0.25      8f
  3  Cu    0.668589  0.668589  -0         8h
  4  Nd    0         0          0         2a

In [106]:
len(analyzer.get_primitive_standard_structure())

13

In [95]:
sites = [
        {'1a': [0., 0., 0.]},
        {'4j': [0.721222, 0.778778, 0.778778],
        '4i': [0.34894, 0.65106, 0.65106]},
        {'4f': [0.5, 0.5, 0. ]}
]
my_crystal.from_random(dim=3, group="I4/mmm", species=['Nd', 'Al', 'Cu'], numIons=[2, 16, 8], sites=sites)

UnboundLocalError: cannot access local variable 'mse' where it is not associated with a value

In [54]:
my_crystal


------Crystal from random------
Dimension: 3
Composition: Nd16
Group: I 4/m m m (139)
 11.6538,  11.6538,   4.5821,  90.0000,  90.0000,  90.0000, tetragonal
Wyckoff sites:
	Nd @ [ 0.0000  0.5000  0.0000], WP [4c] Site [2/m2/m2/m.]
	Nd @ [ 0.0000  0.0000  0.7288], WP [4e] Site m]
	Nd @ [ 0.1832  0.1832  0.0000], WP [8h] Site [m.m2]

In [55]:
# convert to pymatgen structure and visualize
ase_crystal = my_crystal.to_ase()
from ase.visualize import view
view(ase_crystal)

<Popen: returncode: None args: ['/home/kna/.cache/pypoetry/virtualenvs/wycko...>

In [90]:
pyxtal.__version__

AttributeError: type object 'pyxtal' has no attribute '__version__'